# Creating the Tri model (BCR + TLR + NF-kB)

In [1]:
import Base.Threads
Threads.nthreads()
#packages we need
using DifferentialEquations
using Plots 
using CSV
using Distributions
using Random
using DataFrames
using JLD2
using FileIO
using StatsPlots
using Plots.PlotMeasures
using Statistics
plotly();

┌ Warning: PlotlyBase 0.5.4 is not compatible with this version of Plots. The declared compatibility is 0.7.
└ @ Plots /home/simon/.julia/packages/Plots/yJrrq/src/Plots.jl:22


In [2]:
###########################################################################
### File locations  ######################################################
###########################################################################
### set up where CSV2Julia is
locationOfCSV2Julia="CSV2Julia/csv2model-multiscale.py"
#we use the same initial conditions at the PNAS paper 2019 Mitchell Roy et al.
fullModelInitFile="initialConditions/fullModelInits.csv"
moduleDefinitionFilesFolder="moduleDefinitionFiles/"
locationOfFixSpexies="utilityFunctions/"
################################################
### all the packages we need are defined here ##
################################################
import Base.Threads
Threads.nthreads()

12

In [3]:
delay=false

false

In [4]:
#keep this as 1
first_cell=1
#change this if you want more cells
last_cell=25


#total IKK amount in nM, you probably won't change this.
totalIKK=140
#what percentage of IKK should be active during the steady state? Default = 1% = totalIKK/100
basalIKK=totalIKK/100
#you can use the multiplier to scale the IKK during the simualation
IKKMultiplier=1
#this is just a long time to run the steady-state phase for and shouldn't need changing
maxTimeSS=100000.0
#an 8 hour time course = 60*8, you might need longer for non-canonical simulations.
maxTimeTC=60*8
#this is how different the cells should be from each other. leave this at 11% as it is based on our previous work (https://www.pnas.org/doi/10.1073/pnas.1715639115).
preCV=0.11;

#what colours shall we plot with? :seaborn_colorblind is a nice default palette.
colorArray=palette(:seaborn_colorblind);

In [5]:
#this function creates the dynamic ikk time course curve.
function ikkDefault(t,maxTime)
    #explicitely defined input
    IKKparamVals=[0,85,100,85,45,30,25,23,18,15,12,10,10]./100
    paramTime=[0,5,10,15,18,20,300,400,450,480,500,520,max(maxTime,2880)]
    #get the value after the current time point
    indexGTTime=min(searchsortedfirst(paramTime,t),length(paramTime))
    #get the one before
    indexLTTime=max(1,indexGTTime-1)
    timeGTt=paramTime[indexGTTime]
    timeLTt=paramTime[indexLTTime]
    valGTt=IKKparamVals[indexGTTime]
    valLTt=IKKparamVals[indexLTTime]
    timeDiff=timeGTt-timeLTt
    #if the time before and time after are different do a basic linear interpolation between the two.
    if timeDiff>0
        valDiff=valGTt-valLTt
        gradient=valDiff/timeDiff
        timeStep=t-timeLTt
        return valLTt+(gradient*timeStep)
    else
        return valLTt
    end
end


ikkDefault (generic function with 1 method)

In [6]:

#define the 4 variables mentioned above.
ikkSS=t->basalIKK

ikkTC=t->basalIKK+(ikkDefault(t, maxTimeTC)*IKKMultiplier*totalIKK);
ikkTCLess=t->basalIKK+(ikkDefault(t, maxTimeTC)*(IKKMultiplier/2)*totalIKK);
NIKFuncSS=t->1
NIKFuncTC=t->0

#define the 4 variables mentioned above.
ikkSS1=t->basalIKK
ikkSS5=t->basalIKK.*5
ikkSS7p5=t->basalIKK.*7.5
ikkSS10=t->basalIKK.*10
ikkSS100=t->basalIKK.*100

ikkTC1=t->basalIKK.*1+(ikkDefault(t, maxTimeTC)*IKKMultiplier*totalIKK)
ikkTC100=t->basalIKK.*100+(ikkDefault(t, maxTimeTC)*IKKMultiplier*totalIKK)

ikkTCLess=t->basalIKK+(ikkDefault(t, maxTimeTC)*(IKKMultiplier/2)*totalIKK);

NIKFuncSS=t->1
NIKFuncTC=t->0
NIKFuncSS100=t->1/100


p=plot([1:1:maxTimeTC],ikkTC.(1:1:maxTimeTC),label="smooth fit",title="input curve");
xlabel!("time (h)")
ylabel!("active IKK")

In [7]:
function combineModels(modulesToInclude)
    ################################################
    ### Create a combined model from the modules ###
    ################################################
    #first lets tell Julia where the files are

    linkingModuleName="linking"
    combinedModelLocation="combinedModelDefinitionFiles/"
    mkpath(combinedModelLocation)


    #first lets load the linking file so we know to skip any reactions 
    #which are doubly described by the linking file and one of the modules
    linkingReactions=""
    if(delay)
        linkingReactions = DataFrame(CSV.File(moduleDefinitionFilesFolder*linkingModuleName*"/reactions_withDelay.csv",stringtype=String,types=String))
    else
        linkingReactions = DataFrame(CSV.File(moduleDefinitionFilesFolder*linkingModuleName*"/reactions.csv",stringtype=String,types=String))
    end
    linkingParameters = DataFrame(CSV.File(moduleDefinitionFilesFolder*linkingModuleName*"/parameters.csv",stringtype=String,types=String))
    linkingrateLaws = DataFrame(CSV.File(moduleDefinitionFilesFolder*linkingModuleName*"/rateLaws.csv",stringtype=String,types=String))

    #missing values mess things up so replace with empty strings
    replace!(linkingReactions.Substrate, missing => "")
    replace!(linkingReactions.Products, missing => "")

    #we know we need all the linking reactions in the combined model
    reactionsCombined=linkingReactions
    parametersCombined=linkingParameters
    linkingParameters[:,1]=linkingParameters[:,1].*("-"*"linking")
    theseParams=linkingReactions.Parameters
    replacementParams=""
    newParams=theseParams
    thisLine=1
    for param in theseParams
        if ismissing(param)
            thisLine=thisLine+1
        else
            eachParam=split(param," ")
            eachParam=[i*"-"*"linking" for i in eachParam]
            replacementParams=join(eachParam," ")
            newParams[thisLine]=replacementParams
            thisLine=thisLine+1
        end
    end
    linkingReactions.Parameters=newParams
    rateLawsCombined=linkingrateLaws

    # loop through each model
    for mod in modulesToInclude
        print("adding $(mod) module\n")

        thisModule=moduleDefinitionFilesFolder*mod
        #the NFkB file can have delays but the rest do not
        reactionFile=""
        if mod=="NFkB" && delay
            reactionFile=thisModule*"/reactions_withDelay.csv"
        else
            reactionFile=thisModule*"/reactions.csv"
        end
        parameterFile=thisModule*"/parameters.csv"
        rateLawsFile=thisModule*"/rateLaws.csv"

        parameterDF = CSV.read(parameterFile,DataFrame,stringtype=String,types=String)
        rateLawDF = CSV.read(rateLawsFile,DataFrame,stringtype=String,types=String)
        reactionDF = CSV.read(reactionFile,DataFrame,stringtype=String,types=String)
        #not every entry in the reaction database is filled in and missing data can cause a problem
        replace!(reactionDF.Substrate, missing => "")
        replace!(reactionDF.Products, missing => "")

        #keep track of rows to remove
        #these are rows that appear in individual module reaction files 
        #and are replaced by reactions in the linking module.
        rowsToRemove=[]
        thisRow=1
        for row in eachrow(reactionDF)
            row.Substrate=rstrip(row.Substrate,[' '])
            row.Products=rstrip(row.Products,[' '])
            if nrow(linkingReactions[(linkingReactions[!,"Substrate"].==row.Substrate).&(linkingReactions[!,"Products"].==row.Products),:])>0
                print("removing: $(row.Substrate) -> $(row.Products) as it is included in linking file\n")
                push!(rowsToRemove,thisRow)
            end
            thisRow=thisRow+1
        end
        delete!(reactionDF, rowsToRemove)
        print("adding module $(mod) to combined reaction array\n")

        #we need to append a module ID on each parameter name in the reactions file 
        #because some parameters have the same name in multiple modules
        theseParams=reactionDF.Parameters
        replacementParams=""
        newParams=theseParams
        thisLine=1
        for param in theseParams
            eachParam=split(param," ")
            eachParam=[i*"-"*mod for i in eachParam]
            replacementParams=join(eachParam," ")
            newParams[thisLine]=replacementParams
            thisLine=thisLine+1
        end
        reactionDF.Parameters=newParams
        append!(reactionsCombined,reactionDF)

        #do the same, prepending in the parameters file for the combined model
        parameterDF = CSV.read(parameterFile,DataFrame,stringtype=String,types=String)
        rowsToRemove=[]
        thisRow=1
        for row in eachrow(parameterDF)
            if nrow(linkingParameters[(linkingParameters[!,"parameter"].==row.parameter),:])>0
                print("removing: $(row.parameter) as it is included in linking file\n")
                push!(rowsToRemove,thisRow)
#             elseif occursin("p[",row.value)
#                 print("removing: $(row.parameter) as it includes p[t]\n")
#                 push!(rowsToRemove,thisRow)
            end

            thisRow=thisRow+1
        end
        delete!(parameterDF, rowsToRemove)

        theseParams=parameterDF.parameter
        parameterDF.parameter=parameterDF.parameter.*("-"*mod)

        append!(parametersCombined,parameterDF)

        #rate law duplication seems ok and unambiguous for now
        rateLawsDF=CSV.read(rateLawsFile,DataFrame;stringtype=String,types=String)
        append!(rateLawsCombined,rateLawsDF)

        print("finished $(mod)\n\n")
    end
    #write out the combined model
    CSV.write(combinedModelLocation*"reactions.csv", reactionsCombined)
    CSV.write(combinedModelLocation*"parameters.csv", parametersCombined)
    CSV.write(combinedModelLocation*"rateLaws.csv", rateLawsCombined)
    print("finished combining all modules\n")
    return combinedModelLocation
end

combineModels (generic function with 1 method)

In [8]:
combinedModelLocation=combineModels(["BCR","TLR","NFkB"])

adding BCR module
adding module BCR to combined reaction array
finished BCR

adding TLR module
adding module TLR to combined reaction array
finished TLR

adding NFkB module
removing: RelAp50IkBa -> RelAp50 as it is included in linking file
removing: RelAp50IkBb -> RelAp50 as it is included in linking file
removing: RelAp50IkBe -> RelAp50 as it is included in linking file
removing: RelBp50IkBa -> RelBp50 as it is included in linking file
removing: RelBp50IkBb -> RelBp50 as it is included in linking file
removing: RelBp50IkBe -> RelBp50 as it is included in linking file
removing: RelBp52IkBa -> RelBp52 as it is included in linking file
removing: RelBp52IkBb -> RelBp52 as it is included in linking file
removing: RelBp52IkBe -> RelBp52 as it is included in linking file
removing: cRelp50IkBa -> cRelp50 as it is included in linking file
removing: cRelp50IkBb -> cRelp50 as it is included in linking file
removing: cRelp50IkBe -> cRelp50 as it is included in linking file
removing: cRelp52IkBa -

"combinedModelDefinitionFiles/"

In [9]:
parametersFile=combinedModelLocation*"parameters.csv"
reactionsFile=combinedModelLocation*"reactions.csv"
rateLawsFile=combinedModelLocation*"rateLaws.csv"
parametersDF = DataFrame(CSV.File(parametersFile,types=Dict(:parameter=>String, :value=>String)))
originalParams=deepcopy(parametersDF)

thisModelName="odeModel.jl"
thisParamFile=parametersFile
arguments=[reactionsFile, thisParamFile, rateLawsFile,thisModelName]
cmd=`python3 $locationOfCSV2Julia $arguments param`

#lets run csv2julia (requires python to be installed)
run(cmd)

include(thisModelName)
mkpath("distributedModelFiles")
mv(thisModelName,"distributedModelFiles/"*thisModelName, force=true)
println("Model generated for all conditions")

param
Running CSV2JuliaDiffEq with parameters dynamically determined by a variable, re-run with the 5th argument set to 'scan' or 'inline'
Opening combinedModelDefinitionFiles/rateLaws.csv as rate law file
Opening combinedModelDefinitionFiles/parameters.csv as parameters file
Opening combinedModelDefinitionFiles/reactions.csv as reactions file
Model generated for all conditions


In [10]:
include("runSimulation.jl")

runSimulation (generic function with 1 method)

In [17]:
function inputFunc(t)
    if (t>10)&&(t<30)
        return 1
    else
        return 0
    end
end
function inputFuncHigh(t)
    if (t>10)&&(t<30)
        return 1
    else
        return 0.05
    end
end
function inputFuncSS(t)
        return 0    
end
function inputFuncSSHigh(t)
        return 0.05    
end

function inputFuncHigh(t)
    if (t>10)&&(t<30)
        return 100
    else
        return 0
    end
end
function NIKinputFunc(t)
    if (t>10)&&(t<30)
        return 1
    else
        return 1
    end
end
function NIKinputFunc2(t)
    if (t>4*60)
        return 0.05
    else
        return 1
    end
end

bcrSignalSS=inputFuncSS
bcrSignalSSHigh=inputFuncSSHigh
tlrSignalSS=inputFuncSS
bcrSignalTC=inputFunc
bcrSignalTCHigh=inputFuncHigh
tlrSignalTC=inputFuncHigh
nikSignalSS=NIKFuncSS
nikSignalTC=NIKinputFunc2

conditions=[]
paramsToChange=[]
modifyAmount=[]
BCRSSArray=[]
BCRTCArray=[]
TLRSSArray=[]
TLRTCArray=[]
NIKSSArray=[]
NIKTCArray=[]


#U2932 R1 No muts
push!(conditions,"U2932R1_noMuts_TLR")
push!(paramsToChange,["k1_RelASynth-NFkB","basal_RelBSynth-NFkB"])
push!(modifyAmount,[2.0,2.75])
push!(BCRSSArray,bcrSignalSS)
push!(BCRTCArray,bcrSignalSS)
push!(TLRSSArray,tlrSignalSS)
push!(TLRTCArray,tlrSignalTC)
push!(NIKSSArray,nikSignalSS)
push!(NIKTCArray,nikSignalSS)

#U2932 R2 No muts
push!(conditions,"U2932R2_noMuts_TLR")
push!(paramsToChange,["k1_RelASynth-NFkB","basal_RelBSynth-NFkB"])
push!(modifyAmount,[1.0,2.75])
push!(BCRSSArray,bcrSignalSS)
push!(BCRTCArray,bcrSignalSS)
push!(TLRSSArray,tlrSignalSS)
push!(TLRTCArray,tlrSignalTC)
push!(NIKSSArray,nikSignalSS)
push!(NIKTCArray,nikSignalSS)

#U2932 R1 Tak1 Mut
push!(conditions,"U2932R1_TAK1muts_TLR")
push!(paramsToChange,["k1_RelASynth-NFkB","basal_RelBSynth-NFkB","k1_Tp0-BCR","k1_TRAF6mediatedIKKK_offactivation-TLR"])
push!(modifyAmount,[2.0,2.75,10.0,10.0])
push!(BCRSSArray,bcrSignalSS)
push!(BCRTCArray,bcrSignalSS)
push!(TLRSSArray,tlrSignalSS)
push!(TLRTCArray,tlrSignalTC)
push!(NIKSSArray,nikSignalSS)
push!(NIKTCArray,nikSignalSS)

#U2932 R2 Tak1 muts
push!(conditions,"U2932R2_TAK1muts_TLR")
push!(paramsToChange,["k1_RelASynth-NFkB","basal_RelBSynth-NFkB","k1_Tp0-BCR","k1_TRAF6mediatedIKKK_offactivation-TLR"])
push!(modifyAmount,[1.0,2.75,10.0,10.0])
push!(BCRSSArray,bcrSignalSS)
push!(BCRTCArray,bcrSignalSS)
push!(TLRSSArray,tlrSignalSS)
push!(TLRTCArray,tlrSignalTC)
push!(NIKSSArray,nikSignalSS)
push!(NIKTCArray,nikSignalSS)

#HBL1 no muts
push!(conditions,"HBL1_noMuts_TLR")
push!(paramsToChange,["k1_RelASynth-NFkB"])
push!(modifyAmount,[0.6])
push!(BCRSSArray,bcrSignalSS)
push!(BCRTCArray,bcrSignalSS)
push!(TLRSSArray,tlrSignalSS)
push!(TLRTCArray,tlrSignalTC)
push!(NIKSSArray,nikSignalSS)
push!(NIKTCArray,nikSignalSS)

#HBL1 Myd88Muts
push!(conditions,"HBL1_myd88_Muts_TLR")
push!(paramsToChange,["k1_RelASynth-NFkB","k1_MYD88SELFACTIVATION-TLR"])
push!(modifyAmount,[0.6,10])
push!(BCRSSArray,bcrSignalSS)
push!(BCRTCArray,bcrSignalSS)
push!(TLRSSArray,tlrSignalSS)
push!(TLRTCArray,tlrSignalTC)
push!(NIKSSArray,nikSignalSS)
push!(NIKTCArray,nikSignalSS)

#HBL1 CD79B muts
push!(conditions,"HBL1_CD79B_Muts_TLR")
push!(paramsToChange,["k1_RelASynth-NFkB"])
push!(modifyAmount,[0.6])
push!(BCRSSArray,bcrSignalSSHigh)
push!(BCRTCArray,bcrSignalSSHigh)
push!(TLRSSArray,tlrSignalSS)
push!(TLRTCArray,tlrSignalTC)
push!(NIKSSArray,nikSignalSS)
push!(NIKTCArray,nikSignalSS)

#HBL1 Myd88 + CD79B muts
push!(conditions,"HBL1_myd88CD79B_Muts_TLR")
push!(paramsToChange,["k1_RelASynth-NFkB","k1_MYD88SELFACTIVATION-TLR"])
push!(modifyAmount,[0.6,10])
push!(BCRSSArray,bcrSignalSSHigh)
push!(BCRTCArray,bcrSignalSSHigh)
push!(TLRSSArray,tlrSignalSS)
push!(TLRTCArray,tlrSignalTC)
push!(NIKSSArray,nikSignalSS)
push!(NIKTCArray,nikSignalSS)

#RIVA
push!(conditions,"RIVA_TLR_TLR")
push!(paramsToChange,["k1_RelASynth-NFkB"])
push!(modifyAmount,[0.4])
push!(BCRSSArray,bcrSignalSSHigh)
push!(BCRTCArray,bcrSignalSSHigh)
push!(TLRSSArray,tlrSignalSS)
push!(TLRTCArray,tlrSignalTC)
push!(NIKSSArray,nikSignalSS)
push!(NIKTCArray,nikSignalSS)

 println("Summary of conditions being run:")
 folder="outputs"
 show(IOContext(stdout, :limit => false), "text/plain", hcat(conditions,paramsToChange,modifyAmount))


Summary of conditions being run:
9×3 Matrix{Any}:
 "U2932R1_noMuts_TLR"        ["k1_RelASynth-NFkB", "basal_RelBSynth-NFkB"]                                                          [2.0, 2.75]
 "U2932R2_noMuts_TLR"        ["k1_RelASynth-NFkB", "basal_RelBSynth-NFkB"]                                                          [1.0, 2.75]
 "U2932R1_TAK1muts_TLR"      ["k1_RelASynth-NFkB", "basal_RelBSynth-NFkB", "k1_Tp0-BCR", "k1_TRAF6mediatedIKKK_offactivation-TLR"]  [2.0, 2.75, 10.0, 10.0]
 "U2932R2_TAK1muts_TLR"      ["k1_RelASynth-NFkB", "basal_RelBSynth-NFkB", "k1_Tp0-BCR", "k1_TRAF6mediatedIKKK_offactivation-TLR"]  [1.0, 2.75, 10.0, 10.0]
 "HBL1_noMuts_TLR"           ["k1_RelASynth-NFkB"]                                                                                  [0.6]
 "HBL1_myd88_Muts_TLR"       ["k1_RelASynth-NFkB", "k1_MYD88SELFACTIVATION-TLR"]                                                    [0.6, 10.0]
 "HBL1_CD79B_Muts_TLR"       ["k1_RelASynth-NFkB"]                  

In [ ]:
runSimulation(first_cell,last_cell,conditions,folder,BCRSSArray,BCRTCArray,TLRSSArray,TLRTCArray,NIKSSArray,NIKTCArray);;

# Figure 5

In [12]:
include("plotAllSpeciesNormalised.jl")
#toPlot=["U2932R1_noMuts_TLR","U2932R2_noMuts_TLR","U2932R1_CD79Muts_TLR","U2932R2_CD79Muts_TLR","HBL1_noMuts_TLR","HBL1_myd88_Muts_TLR","HBL1_CD79B_Muts_TLR","HBL1_myd88CD79B_Muts_TLR","RIVA_TLR"]
toPlot=["RIVA_TLR","U2932R1_noMuts_TLR","U2932R2_noMuts_TLR","HBL1_noMuts_TLR",]
hoursToPlot=4
thisColorArray=colorArray[[1,3,2,10]]
plotAllSpeciesNormalised(["RelAnp50n"],toPlot,thisColorArray,first_cell,last_cell,folder,hoursToPlot,7,1)


In [13]:
include("plotAllSpeciesNormalised.jl")
#toPlot=["U2932R1_noMuts_TLR","U2932R2_noMuts_TLR","U2932R1_CD79Muts_TLR","U2932R2_CD79Muts_TLR","HBL1_noMuts_TLR","HBL1_myd88_Muts_TLR","HBL1_CD79B_Muts_TLR","HBL1_myd88CD79B_Muts_TLR","RIVA_TLR"]
toPlot=["HBL1_noMuts_TLR","HBL1_myd88_Muts_TLR","HBL1_CD79B_Muts_TLR","HBL1_myd88CD79B_Muts_TLR",]
hoursToPlot=4
thisColorArray=colorArray[[10,5,7,8]]
plotAllSpeciesNormalised(["RelAnp50n"],toPlot,thisColorArray,first_cell,last_cell,folder,hoursToPlot,7,1)

In [14]:
include("plotAllSpeciesNormalised.jl")
#toPlot=["U2932R1_noMuts_TLR","U2932R2_noMuts_TLR","U2932R1_CD79Muts_TLR","U2932R2_CD79Muts_TLR","HBL1_noMuts_TLR","HBL1_myd88_Muts_TLR","HBL1_CD79B_Muts_TLR","HBL1_myd88CD79B_Muts_TLR","RIVA_TLR"]
toPlot=["U2932R1_noMuts_TLR","U2932R2_noMuts_TLR","U2932R1_TAK1muts_TLR","U2932R2_TAK1muts_TLR"]
hoursToPlot=4
thisColorArray=vcat(colorArray[[3,2,8]],RGB(0.0,0.0,0.0))
plotAllSpeciesNormalised(["RelAnp50n"],toPlot,thisColorArray,first_cell,last_cell,folder,hoursToPlot,7,1)


In [15]:
include("plotAllSpeciesNormalised.jl")
#toPlot=["U2932R1_noMuts_TLR","U2932R2_noMuts_TLR","U2932R1_CD79Muts_TLR","U2932R2_CD79Muts_TLR","HBL1_noMuts_TLR","HBL1_myd88_Muts_TLR","HBL1_CD79B_Muts_TLR","HBL1_myd88CD79B_Muts_TLR","RIVA_TLR"]
toPlot=["RIVA_TLR","U2932R1_TAK1muts_TLR","U2932R2_TAK1muts_TLR","HBL1_myd88CD79B_Muts_TLR"]
hoursToPlot=4
thisColorArray=colorArray[[1,3,2,10]]
plotAllSpeciesNormalised(["RelAnp50n"],toPlot,thisColorArray,first_cell,last_cell,folder,hoursToPlot,7,1)
